In [1]:
import graphlab

#Read some product review data from a preloaded folder

In [2]:
products = graphlab.SFrame('amazon_baby.gl/')

[INFO] This non-commercial license of GraphLab Create is assigned to akorotcov@yahoo.comand will expire on September 23, 2016. For commercial licensing options, visit https://dato.com/buy/.

[INFO] Start server at: ipc:///tmp/graphlab_server-37120 - Server binary: /Users/Alex/.graphlab/anaconda/lib/python2.7/site-packages/graphlab/unity_server - Server log: /tmp/graphlab_server_1443918615.log
[INFO] GraphLab Server Version: 1.6.1


#Let's explore the data

In [4]:
products.head()

Columns:
	name	str
	review	str
	rating	float

Rows: 10

Data:
+-------------------------------+-------------------------------+--------+
|              name             |             review            | rating |
+-------------------------------+-------------------------------+--------+
|    Planetwise Flannel Wipes   | These flannel wipes are OK... |  3.0   |
|     Planetwise Wipe Pouch     | it came early and was not ... |  5.0   |
| Annas Dream Full Quilt wit... | Very soft and comfortable ... |  5.0   |
| Stop Pacifier Sucking with... | This is a product well wor... |  5.0   |
| Stop Pacifier Sucking with... | All of my kids have cried ... |  5.0   |
| Stop Pacifier Sucking with... | When the Binky Fairy came ... |  5.0   |
| A Tale of Baby's Days with... | Lovely book, it's bound ti... |  4.0   |
| Baby Tracker&reg; - Daily ... | Perfect for new parents. W... |  5.0   |
| Baby Tracker&reg; - Daily ... | A friend of mine pinned th... |  5.0   |
| Baby Tracker&reg; - Daily ... | This has been an easy way ... |  4.0   |
+-------------------------------+-------------------------------+--------+
[10 rows x 3 columns]

#Build word count vector for each review

In [5]:
products['word_count'] = graphlab.text_analytics.count_words(products['review'])

In [6]:
products.head()

Columns:
	name	str
	review	str
	rating	float
	word_count	dict

Rows: 10

Data:
+-------------------------------+-------------------------------+--------+
|              name             |             review            | rating |
+-------------------------------+-------------------------------+--------+
|    Planetwise Flannel Wipes   | These flannel wipes are OK... |  3.0   |
|     Planetwise Wipe Pouch     | it came early and was not ... |  5.0   |
| Annas Dream Full Quilt wit... | Very soft and comfortable ... |  5.0   |
| Stop Pacifier Sucking with... | This is a product well wor... |  5.0   |
| Stop Pacifier Sucking with... | All of my kids have cried ... |  5.0   |
| Stop Pacifier Sucking with... | When the Binky Fairy came ... |  5.0   |
| A Tale of Baby's Days with... | Lovely book, it's bound ti... |  4.0   |
| Baby Tracker&reg; - Daily ... | Perfect for new parents. W... |  5.0   |
| Baby Tracker&reg; - Daily ... | A friend of mine pinned th... |  5.0   |
| Baby Tracker&reg; - Daily ... | This has been an easy way ... |  4.0   |
+-------------------------------+-------------------------------+--------+
+-------------------------------+
|           word_count          |
+-------------------------------+
| {'and': 5, '6': 1, 'stink'... |
| {'and': 3, 'love': 1, 'it'... |
| {'and': 2, 'quilt': 1, 'it... |
| {'ingenious': 1, 'and': 3,... |
| {'and': 2, 'parents!!': 1,... |
| {'and': 2, 'this': 2, 'her... |
| {'shop': 1, 'noble': 1, 'i... |
| {'and': 2, 'all': 1, 'righ... |
| {'and': 1, 'help': 1, 'giv... |
| {'journal.': 1, 'nanny': 1... |
+-------------------------------+
[10 rows x 4 columns]

In [8]:
graphlab.canvas.set_target('ipynb')

In [10]:
products['name'].show()

#Let's explore Vulli Sophie

In [11]:
giraffe_reviews = products[products['name'] == 'Vulli Sophie the Giraffe Teether']

In [12]:
len(giraffe_reviews)

785

In [13]:
giraffe_reviews['rating'].show(view='Categorical')

#Build a sentiment classifier

In [15]:
products['rating'].show(view='Categorical')

##Define what is positive and negative sentiment

In [16]:
# Remove all 3* ratings
products_new = products[products['rating'] != 3]

In [17]:
# psitive sentiment for 4* and 5* reviews
products_new['sentiment'] = products_new['rating'] >= 4

In [19]:
products_new.head()

Columns:
	name	str
	review	str
	rating	float
	word_count	dict
	sentiment	int

Rows: 10

Data:
+-------------------------------+-------------------------------+--------+
|              name             |             review            | rating |
+-------------------------------+-------------------------------+--------+
|     Planetwise Wipe Pouch     | it came early and was not ... |  5.0   |
| Annas Dream Full Quilt wit... | Very soft and comfortable ... |  5.0   |
| Stop Pacifier Sucking with... | This is a product well wor... |  5.0   |
| Stop Pacifier Sucking with... | All of my kids have cried ... |  5.0   |
| Stop Pacifier Sucking with... | When the Binky Fairy came ... |  5.0   |
| A Tale of Baby's Days with... | Lovely book, it's bound ti... |  4.0   |
| Baby Tracker&reg; - Daily ... | Perfect for new parents. W... |  5.0   |
| Baby Tracker&reg; - Daily ... | A friend of mine pinned th... |  5.0   |
| Baby Tracker&reg; - Daily ... | This has been an easy way ... |  4.0   |
| Baby Tracker&reg; - Daily ... | I love this journal and ou... |  4.0   |
+-------------------------------+-------------------------------+--------+
+-------------------------------+-----------+
|           word_count          | sentiment |
+-------------------------------+-----------+
| {'and': 3, 'love': 1, 'it'... |     1     |
| {'and': 2, 'quilt': 1, 'it... |     1     |
| {'ingenious': 1, 'and': 3,... |     1     |
| {'and': 2, 'parents!!': 1,... |     1     |
| {'and': 2, 'this': 2, 'her... |     1     |
| {'shop': 1, 'noble': 1, 'i... |     1     |
| {'and': 2, 'all': 1, 'righ... |     1     |
| {'and': 1, 'help': 1, 'giv... |     1     |
| {'journal.': 1, 'nanny': 1... |     1     |
| {'all': 1, 'forget': 1, 'j... |     1     |
+-------------------------------+-----------+
[10 rows x 5 columns]

#Let's train the classifier

In [20]:
train_data, test_data = products_new.random_split(.8, seed = 0)

In [21]:
sentiment_model = graphlab.logistic_classifier.create(train_data, target='sentiment', 
                                                      features=['word_count'], 
                                                      validation_set = test_data)

PROGRESS: Logistic regression:
PROGRESS: --------------------------------------------------------
PROGRESS: Number of examples          : 133448
PROGRESS: Number of classes           : 2
PROGRESS: Number of feature columns   : 1
PROGRESS: Number of unpacked features : 219217
PROGRESS: Number of coefficients    : 219218
PROGRESS: Starting L-BFGS
PROGRESS: --------------------------------------------------------
PROGRESS: +-----------+----------+-----------+--------------+-------------------+---------------------+
PROGRESS: | Iteration | Passes   | Step size | Elapsed Time | Training-accuracy | Validation-accuracy |
PROGRESS: +-----------+----------+-----------+--------------+-------------------+---------------------+
PROGRESS: | 1         | 5        | 0.000002  | 2.620693     | 0.841481          | 0.839989            |
PROGRESS: | 2         | 9        | 3.000000  | 4.242353     | 0.947425          | 0.894877            |
PROGRESS: | 3         | 10       | 3.000000  | 4.903379     | 0.92

#Evaluate a sentiment model

In [22]:
sentiment_model.evaluate(test_data, metric='roc_curve')

{'roc_curve': Columns:
 	threshold	float
 	fpr	float
 	tpr	float
 	p	int
 	n	int
 
 Rows: 1001
 
 Data:
 +------------------+----------------+------------------+-------+------+
 |    threshold     |      fpr       |       tpr        |   p   |  n   |
 +------------------+----------------+------------------+-------+------+
 |       0.0        | 0.21732608288  | 0.00574712643678 | 28014 | 5333 |
 | 0.0010000000475  | 0.78267391712  |  0.994252873563  | 28014 | 5333 |
 | 0.00200000009499 | 0.743858991187 |  0.992967801813  | 28014 | 5333 |
 | 0.00300000002608 | 0.721920120008 |  0.992182480188  | 28014 | 5333 |
 | 0.00400000018999 | 0.707481717607 |  0.991468551439  | 28014 | 5333 |
 | 0.00499999988824 | 0.696606037877 |  0.990968801314  | 28014 | 5333 |
 | 0.00600000005215 | 0.687042940184 |  0.990433354751  | 28014 | 5333 |
 | 0.00700000021607 | 0.677292330771 |  0.989933604626  | 28014 | 5333 |
 | 0.00800000037998 | 0.667166697919 |  0.989612336689  | 28014 | 5333 |
 | 0.00899999961257 

In [24]:
sentiment_model.show(view='Evaluation')

#Applying the model to understand sentiment the Giraffe

In [26]:
giraffe_reviews['predicted_sentiment'] = sentiment_model.predict(giraffe_reviews, output_type='probability')

In [27]:
giraffe_reviews.head()

Columns:
	name	str
	review	str
	rating	float
	word_count	dict
	predicted_sentiment	float

Rows: 10

Data:
+-------------------------------+-------------------------------+--------+
|              name             |             review            | rating |
+-------------------------------+-------------------------------+--------+
| Vulli Sophie the Giraffe T... | He likes chewing on all th... |  5.0   |
| Vulli Sophie the Giraffe T... | My son loves this toy and ... |  5.0   |
| Vulli Sophie the Giraffe T... | There really should be a l... |  1.0   |
| Vulli Sophie the Giraffe T... | All the moms in my moms' g... |  5.0   |
| Vulli Sophie the Giraffe T... | I was a little skeptical o... |  5.0   |
| Vulli Sophie the Giraffe T... | I have been reading about ... |  5.0   |
| Vulli Sophie the Giraffe T... | My neice loves her sophie ... |  5.0   |
| Vulli Sophie the Giraffe T... | What a friendly face!  And... |  5.0   |
| Vulli Sophie the Giraffe T... | We got this just for my so... |  5.0   |
| Vulli Sophie the Giraffe T... | My baby seems to like this... |  3.0   |
+-------------------------------+-------------------------------+--------+
+-------------------------------+---------------------+
|           word_count          | predicted_sentiment |
+-------------------------------+---------------------+
| {'and': 1, 'all': 1, 'beca... |    0.999513023521   |
| {'and': 1, 'right': 1, 'he... |    0.999320678306   |
| {'and': 2, 'all': 1, 'woul... |    0.013558811687   |
| {'and': 2, 'one!': 1, 'all... |    0.995769474148   |
| {'and': 3, 'all': 1, 'mont... |    0.662374415673   |
| {'and': 6, 'seven': 1, 'al... |    0.999997148186   |
| {'and': 4, 'drooling,': 1,... |    0.989190989536   |
| {'and': 3, 'chew': 1, 'be'... |    0.999563518413   |
| {'chew': 2, 'seemed': 1, '... |    0.970160542725   |
| {'and': 2, 'already': 1, '... |    0.195367644588   |
+-------------------------------+---------------------+
[10 rows x 5 columns]

##Sort the reviews based on pedicted sentiment and explore

In [30]:
giraffe_reviews = giraffe_reviews.sort('predicted_sentiment', ascending=False)

In [32]:
giraffe_reviews.head()

Columns:
	name	str
	review	str
	rating	float
	word_count	dict
	predicted_sentiment	float

Rows: 10

Data:
+-------------------------------+-------------------------------+--------+
|              name             |             review            | rating |
+-------------------------------+-------------------------------+--------+
| Vulli Sophie the Giraffe T... | Sophie, oh Sophie, your ti... |  5.0   |
| Vulli Sophie the Giraffe T... | I'm not sure why Sophie is... |  4.0   |
| Vulli Sophie the Giraffe T... | I'll be honest...I bought ... |  4.0   |
| Vulli Sophie the Giraffe T... | We got this little giraffe... |  5.0   |
| Vulli Sophie the Giraffe T... | As a mother of 16month old... |  5.0   |
| Vulli Sophie the Giraffe T... | Sophie the Giraffe is the ... |  5.0   |
| Vulli Sophie the Giraffe T... | Sophie la giraffe is absol... |  5.0   |
| Vulli Sophie the Giraffe T... | My 5-mos old son took to t... |  5.0   |
| Vulli Sophie the Giraffe T... | My nephews and my four kid... |  5.0   |
| Vulli Sophie the Giraffe T... | Never thought I'd see my s... |  5.0   |
+-------------------------------+-------------------------------+--------+
+-------------------------------+---------------------+
|           word_count          | predicted_sentiment |
+-------------------------------+---------------------+
| {'giggles': 1, 'all': 1, "... |         1.0         |
| {'adoring': 1, 'find': 1, ... |    0.999999999703   |
| {'all': 2, 'discovered': 1... |    0.999999999392   |
| {'all': 2, "don't": 1, '(l... |    0.99999999919    |
| {'cute': 1, 'all': 1, 'rev... |    0.999999998657   |
| {'just': 2, 'both': 1, 'mo... |    0.999999997108   |
| {'and': 5, 'the': 1, 'all'... |    0.999999995589   |
| {'just': 1, 'shape': 2, 'm... |    0.999999995573   |
| {'and': 4, 'chew': 1, 'all... |    0.999999989527   |
| {'giggles': 1, 'all': 1, '... |    0.999999985069   |
+-------------------------------+---------------------+
[10 rows x 5 columns]

In [33]:
giraffe_reviews[0]['review']

"Sophie, oh Sophie, your time has come. My granddaughter, Violet is 5 months old and starting to teeth. What joy little Sophie brings to Violet. Sophie is made of a very pliable rubber that is sturdy but not tough. It is quite easy for Violet to twist Sophie into unheard of positions to get Sophie into her mouth. The little nose and hooves fit perfectly into small mouths, and the drooling has purpose. The paint on Sophie is food quality.Sophie was born in 1961 in France. The maker had wondered why there was nothing available for babies and made Sophie from the finest rubber, phthalate-free on St Sophie's Day, thus the name was born. Since that time millions of Sophie's populate the world. She is soft and for babies little hands easy to grasp. Violet especially loves the bumpy head and horns of Sophie. Sophie has a long neck that easy to grasp and twist. She has lovely, sizable spots that attract Violet's attention. Sophie has happy little squeaks that bring squeals of delight from Viol

In [34]:
giraffe_reviews[1]['review']

"I'm not sure why Sophie is such a hit with the little ones, but my 7 month old baby girl is one of her adoring fans.  The rubber is softer and more pleasant to handle, and my daughter has enjoyed chewing on her legs and the nubs on her head even before she started teething.  She also loves the squeak that Sophie makes when you squeeze her.  Not sure what it is but if Sophie is amongst a pile of her other toys, my daughter will more often than not reach for Sophie.  And I have the peace of mind of knowing that only edible and safe paints and materials have been used to make Sophie, as opposed to Bright Starts and other baby toys made in China.  Now that the research is out on phthalates and other toxic substances in baby toys, I think it's more important than ever to find good quality toys that are also safe for our babies to handle and put in their mouths.  Sophie is a must-have for every new mom in my opinion.  Even if your kid is one of the few that can take or leave her, it's worth

##Show most negative review

In [35]:
giraffe_reviews[-1]['review']

"My son (now 2.5) LOVED his Sophie, and I bought one for every baby shower I've gone to. Now, my daughter (6 months) just today nearly choked on it and I will never give it to her again. Had I not been within hearing range it could have been fatal. The strange sound she was making caught my attention and when I went to her and found the front curved leg shoved well down her throat and her face a purply/blue I panicked. I pulled it out and she vomited all over the carpet before screaming her head off. I can't believe how my opinion of this toy has changed from a must-have to a must-not-use. Please don't disregard any of the choking hazard comments, they are not over exaggerated!"

In [37]:
giraffe_reviews[-2]['review']

"This children's toy is nostalgic and very cute. However, there is a distinct rubber smell and a very odd taste, yes I tried it, that my baby did not enjoy. Also, if it is soiled it is extremely difficult to clean as the rubber is a kind of porus material and does not clean well. The final thing is the squeaking device inside which stopped working after the first couple of days. I returned this item feeling I had overpaid for a toy that was defective and did not meet my expectations. Please do not be swayed by the cute packaging and hype surounding it as I was. One more thing, I was given a full refund from Amazon without any problem."

#Analyzing product sentiment - Home Work

##1. Use .apply() to build a new feature with the counts for each of the selected_words

In [39]:
# In the IPython notebook above, we used the word counts for all words in the reviews 
# to train the sentiment classifier model. Now, we are going to follow a similar path, 
# but only use this subset of the words:
selected_words = ['awesome', 'great', 'fantastic', 'amazing', 'love', 'horrible', 
                  'bad', 'terrible', 'awful', 'wow', 'hate']

In [50]:
#You will write a function called awesome_count which takes in the word counts and returns 
#the number of times ‘awesome’ appears in the reviews.
def awesome_count(word_count, selected_words=selected_words):
    """This function return a dictionary contaning slected_words 
    and counts of thess words in the word_count dictionary"""
    awesome_dic = {}
    for word in selected_words:
        if word in word_count:
            awesome_dic[word] = word_count[word] 
        else:
            awesome_dic[word] = 0
    return awesome_dic

#word_count = giraffe_reviews[2]['word_count']
#print 'word_count out', word_count
#print awesome_count(word_count)

In [53]:
products_new['selected_words'] = products_new['word_count'].apply(awesome_count)

In [56]:
#products_new.remove_column('awesome')
products_new.head()

Columns:
	name	str
	review	str
	rating	float
	word_count	dict
	sentiment	int
	selected_words	dict

Rows: 10

Data:
+-------------------------------+-------------------------------+--------+
|              name             |             review            | rating |
+-------------------------------+-------------------------------+--------+
|     Planetwise Wipe Pouch     | it came early and was not ... |  5.0   |
| Annas Dream Full Quilt wit... | Very soft and comfortable ... |  5.0   |
| Stop Pacifier Sucking with... | This is a product well wor... |  5.0   |
| Stop Pacifier Sucking with... | All of my kids have cried ... |  5.0   |
| Stop Pacifier Sucking with... | When the Binky Fairy came ... |  5.0   |
| A Tale of Baby's Days with... | Lovely book, it's bound ti... |  4.0   |
| Baby Tracker&reg; - Daily ... | Perfect for new parents. W... |  5.0   |
| Baby Tracker&reg; - Daily ... | A friend of mine pinned th... |  5.0   |
| Baby Tracker&reg; - Daily ... | This has been an easy way ... |  4.0   |
| Baby Tracker&reg; - Daily ... | I love this journal and ou... |  4.0   |
+-------------------------------+-------------------------------+--------+
+-------------------------------+-----------+-------------------------------+
|           word_count          | sentiment |         selected_words        |
+-------------------------------+-----------+-------------------------------+
| {'and': 3, 'love': 1, 'it'... |     1     | {'great': 0, 'love': 1, 'a... |
| {'and': 2, 'quilt': 1, 'it... |     1     | {'great': 0, 'love': 0, 'a... |
| {'ingenious': 1, 'and': 3,... |     1     | {'great': 0, 'love': 2, 'a... |
| {'and': 2, 'parents!!': 1,... |     1     | {'great': 1, 'love': 0, 'a... |
| {'and': 2, 'this': 2, 'her... |     1     | {'great': 1, 'love': 0, 'a... |
| {'shop': 1, 'noble': 1, 'i... |     1     | {'great': 0, 'love': 0, 'a... |
| {'and': 2, 'all': 1, 'righ... |     1     | {'great': 0, 'love': 0, 'a... |
| {'and': 1, 'help': 1, 'giv... |     1     | {'great': 0, 'love': 0, 'a... |
| {'journal.': 1, 'nanny': 1... |     1     | {'great': 0, 'love': 0, 'a... |
| {'all': 1, 'forget': 1, 'j... |     1     | {'great': 0, 'love': 2, 'a... |
+-------------------------------+-----------+-------------------------------+
[10 rows x 6 columns]

In [57]:
#Create new columns with titles as a selected_word and counts for each word
for word in selected_words:
    products_new[word] = products_new['selected_words'].apply(lambda x: x[word])

products_new.head()

Columns:
	name	str
	review	str
	rating	float
	word_count	dict
	sentiment	int
	selected_words	dict
	awesome	int
	great	int
	fantastic	int
	amazing	int
	love	int
	horrible	int
	bad	int
	terrible	int
	awful	int
	wow	int
	hate	int

Rows: 10

Data:
+-------------------------------+-------------------------------+--------+
|              name             |             review            | rating |
+-------------------------------+-------------------------------+--------+
|     Planetwise Wipe Pouch     | it came early and was not ... |  5.0   |
| Annas Dream Full Quilt wit... | Very soft and comfortable ... |  5.0   |
| Stop Pacifier Sucking with... | This is a product well wor... |  5.0   |
| Stop Pacifier Sucking with... | All of my kids have cried ... |  5.0   |
| Stop Pacifier Sucking with... | When the Binky Fairy came ... |  5.0   |
| A Tale of Baby's Days with... | Lovely book, it's bound ti... |  4.0   |
| Baby Tracker&reg; - Daily ... | Perfect for new parents. W... |  5.0   |
| Baby Tracker&reg; - Daily ... | A friend of mine pinned th... |  5.0   |
| Baby Tracker&reg; - Daily ... | This has been an easy way ... |  4.0   |
| Baby Tracker&reg; - Daily ... | I love this journal and ou... |  4.0   |
+-------------------------------+-------------------------------+--------+
+-------------------------------+-----------+-------------------------------+---------+
|           word_count          | sentiment |         selected_words        | awesome |
+-------------------------------+-----------+-------------------------------+---------+
| {'and': 3, 'love': 1, 'it'... |     1     | {'great': 0, 'love': 1, 'a... |    0    |
| {'and': 2, 'quilt': 1, 'it... |     1     | {'great': 0, 'love': 0, 'a... |    0    |
| {'ingenious': 1, 'and': 3,... |     1     | {'great': 0, 'love': 2, 'a... |    0    |
| {'and': 2, 'parents!!': 1,... |     1     | {'great': 1, 'love': 0, 'a... |    0    |
| {'and': 2, 'this': 2, 'her... |     1     | {'great': 1, 'love': 0, 'a... |    0    |
| {'shop': 1, 'noble': 1, 'i... |     1     | {'great': 0, 'love': 0, 'a... |    0    |
| {'and': 2, 'all': 1, 'righ... |     1     | {'great': 0, 'love': 0, 'a... |    0    |
| {'and': 1, 'help': 1, 'giv... |     1     | {'great': 0, 'love': 0, 'a... |    0    |
| {'journal.': 1, 'nanny': 1... |     1     | {'great': 0, 'love': 0, 'a... |    0    |
| {'all': 1, 'forget': 1, 'j... |     1     | {'great': 0, 'love': 2, 'a... |    0    |
+-------------------------------+-----------+-------------------------------+---------+
+-------+-----------+---------+------+----------+-----+----------+-------+-----+
| great | fantastic | amazing | love | horrible | bad | terrible | awful | wow |
+-------+-----------+---------+------+----------+-----+----------+-------+-----+
|   0   |     0     |    0    |  1   |    0     |  0  |    0     |   0   |  0  |
|   0   |     0     |    0    |  0   |    0     |  0  |    0     |   0   |  0  |
|   0   |     0     |    0    |  2   |    0     |  0  |    0     |   0   |  0  |
|   1   |     0     |    0    |  0   |    0     |  0  |    0     |   0   |  0  |
|   1   |     0     |    0    |  0   |    0     |  0  |    0     |   0   |  0  |
|   0   |     0     |    0    |  0   |    0     |  0  |    0     |   0   |  0  |
|   0   |     0     |    0    |  0   |    0     |  0  |    0     |   0   |  0  |
|   0   |     0     |    0    |  0   |    0     |  0  |    0     |   0   |  0  |
|   0   |     0     |    0    |  0   |    0     |  0  |    0     |   0   |  0  |
|   0   |     0     |    0    |  2   |    0     |  0  |    0     |   0   |  0  |
+-------+-----------+---------+------+----------+-----+----------+-------+-----+
+------+
| hate |
+------+
|  0   |
|  0   |
|  0   |
|  0   |
|  0   |
|  0   |
|  0   |
|  0   |
|  0   |
|  0   |
+------+
[10 rows x 17 columns]

In [59]:
# Using the .sum() method on each of the new columns you created, 
# answer the following questions: Out of the selected_words, 
# which one is most used in the dataset? Which one is least used?
stat_selected_words = {}
for word in selected_words:
    stat_selected_words[word] = products_new[word].sum()

print stat_selected_words

{'fantastic': 873, 'love': 40277, 'bad': 3197, 'awesome': 2002, 'great': 42420, 'terrible': 673, 'amazing': 1305, 'horrible': 659, 'awful': 345, 'hate': 1057, 'wow': 131}


In [67]:
from collections import OrderedDict
ordered_stat = OrderedDict(sorted(stat_selected_words.items(), key=lambda t: t[1]))

##Most and leasr used words

In [72]:
#print stat_selected_words
#print ordered_stat
print 'Most used word is:', ordered_stat.keys()[-1] 
print 'List used word is:', ordered_stat.keys()[0] 

Most used word is: great
List used word is: wow


#2. Create a new sentiment analysis model using only the selected_words as features

In [73]:
# Use the same train/test split as in the IPython Notebook from lecture
train_data1,test_data1 = products_new.random_split(.8, seed=0)

In [74]:
train_data1.head()

Columns:
	name	str
	review	str
	rating	float
	word_count	dict
	sentiment	int
	selected_words	dict
	awesome	int
	great	int
	fantastic	int
	amazing	int
	love	int
	horrible	int
	bad	int
	terrible	int
	awful	int
	wow	int
	hate	int

Rows: 10

Data:
+-------------------------------+-------------------------------+--------+
|              name             |             review            | rating |
+-------------------------------+-------------------------------+--------+
|     Planetwise Wipe Pouch     | it came early and was not ... |  5.0   |
| Annas Dream Full Quilt wit... | Very soft and comfortable ... |  5.0   |
| Stop Pacifier Sucking with... | This is a product well wor... |  5.0   |
| Stop Pacifier Sucking with... | All of my kids have cried ... |  5.0   |
| Stop Pacifier Sucking with... | When the Binky Fairy came ... |  5.0   |
| A Tale of Baby's Days with... | Lovely book, it's bound ti... |  4.0   |
| Baby Tracker&reg; - Daily ... | Perfect for new parents. W... |  5.0   |
| Baby Tracker&reg; - Daily ... | A friend of mine pinned th... |  5.0   |
| Baby Tracker&reg; - Daily ... | This has been an easy way ... |  4.0   |
| Baby Tracker&reg; - Daily ... | I love this journal and ou... |  4.0   |
+-------------------------------+-------------------------------+--------+
+-------------------------------+-----------+-------------------------------+---------+
|           word_count          | sentiment |         selected_words        | awesome |
+-------------------------------+-----------+-------------------------------+---------+
| {'and': 3, 'love': 1, 'it'... |     1     | {'great': 0, 'love': 1, 'a... |    0    |
| {'and': 2, 'quilt': 1, 'it... |     1     | {'great': 0, 'love': 0, 'a... |    0    |
| {'ingenious': 1, 'and': 3,... |     1     | {'great': 0, 'love': 2, 'a... |    0    |
| {'and': 2, 'parents!!': 1,... |     1     | {'great': 1, 'love': 0, 'a... |    0    |
| {'and': 2, 'this': 2, 'her... |     1     | {'great': 1, 'love': 0, 'a... |    0    |
| {'shop': 1, 'noble': 1, 'i... |     1     | {'great': 0, 'love': 0, 'a... |    0    |
| {'and': 2, 'all': 1, 'righ... |     1     | {'great': 0, 'love': 0, 'a... |    0    |
| {'and': 1, 'help': 1, 'giv... |     1     | {'great': 0, 'love': 0, 'a... |    0    |
| {'journal.': 1, 'nanny': 1... |     1     | {'great': 0, 'love': 0, 'a... |    0    |
| {'all': 1, 'forget': 1, 'j... |     1     | {'great': 0, 'love': 2, 'a... |    0    |
+-------------------------------+-----------+-------------------------------+---------+
+-------+-----------+---------+------+----------+-----+----------+-------+-----+
| great | fantastic | amazing | love | horrible | bad | terrible | awful | wow |
+-------+-----------+---------+------+----------+-----+----------+-------+-----+
|   0   |     0     |    0    |  1   |    0     |  0  |    0     |   0   |  0  |
|   0   |     0     |    0    |  0   |    0     |  0  |    0     |   0   |  0  |
|   0   |     0     |    0    |  2   |    0     |  0  |    0     |   0   |  0  |
|   1   |     0     |    0    |  0   |    0     |  0  |    0     |   0   |  0  |
|   1   |     0     |    0    |  0   |    0     |  0  |    0     |   0   |  0  |
|   0   |     0     |    0    |  0   |    0     |  0  |    0     |   0   |  0  |
|   0   |     0     |    0    |  0   |    0     |  0  |    0     |   0   |  0  |
|   0   |     0     |    0    |  0   |    0     |  0  |    0     |   0   |  0  |
|   0   |     0     |    0    |  0   |    0     |  0  |    0     |   0   |  0  |
|   0   |     0     |    0    |  2   |    0     |  0  |    0     |   0   |  0  |
+-------+-----------+---------+------+----------+-----+----------+-------+-----+
+------+
| hate |
+------+
|  0   |
|  0   |
|  0   |
|  0   |
|  0   |
|  0   |
|  0   |
|  0   |
|  0   |
|  0   |
+------+
[10 rows x 17 columns]

In [75]:
# Train a logistic regression classifier (use graphlab.logistic_classifier.create) using just the selected_words. 
# Hint: you can use this parameter in the .create() call to specify the features used to be exactly 
# the new columns you just created:
# features=selected_words
# Call your new model: selected_words_model
selected_words_model = graphlab.logistic_classifier.create(train_data1, target = 'sentiment', 
                                                      features = selected_words, 
                                                      validation_set = test_data1)


PROGRESS: Logistic regression:
PROGRESS: --------------------------------------------------------
PROGRESS: Number of examples          : 133448
PROGRESS: Number of classes           : 2
PROGRESS: Number of feature columns   : 11
PROGRESS: Number of unpacked features : 11
PROGRESS: Number of coefficients    : 12
PROGRESS: Starting Newton Method
PROGRESS: --------------------------------------------------------
PROGRESS: +-----------+----------+--------------+-------------------+---------------------+
PROGRESS: | Iteration | Passes   | Elapsed Time | Training-accuracy | Validation-accuracy |
PROGRESS: +-----------+----------+--------------+-------------------+---------------------+
PROGRESS: | 1         | 2        | 0.211180     | 0.844299          | 0.842842            |
PROGRESS: | 2         | 3        | 0.347536     | 0.844186          | 0.842842            |
PROGRESS: | 3         | 4        | 0.473970     | 0.844276          | 0.843142            |
PROGRESS: | 4         | 5        |

##Examine the weights the learned classifier assigned to each of the 11 words

In [101]:
# GraphLab Create, a learned model, such as the selected_words_model, 
# has a field 'coefficients', which lets you look at the learned coefficients.
#  Out of the 11 words in selected_words, which one got the most positive weight? 
# Which one got the most negative weight? Do these values make sense for you? 

selected_words_model['coefficients'].sort('value', ascending=False).print_rows(num_rows=12, num_columns=4)


+-------------+-------+-------+------------------+
|     name    | index | class |      value       |
+-------------+-------+-------+------------------+
|     love    |  None |   1   |  1.39989834302   |
| (intercept) |  None |   1   |  1.36728315229   |
|   awesome   |  None |   1   |  1.05800888878   |
|   amazing   |  None |   1   |  0.892802422508  |
|  fantastic  |  None |   1   |  0.891303090304  |
|    great    |  None |   1   |  0.883937894898  |
|     wow     |  None |   1   | -0.0541450123333 |
|     bad     |  None |   1   | -0.985827369929  |
|     hate    |  None |   1   |  -1.40916406276  |
|    awful    |  None |   1   |  -1.76469955631  |
|   horrible  |  None |   1   |  -1.99651800559  |
|   terrible  |  None |   1   |  -2.09049998487  |
+-------------+-------+-------+------------------+
[12 rows x 4 columns]



#3. Comparing the accuracy of different sentiment analysis model: Using the method

##.evaluate(test_data)

In [83]:
# What is the accuracy of the selected_words_model on the test_data? 
# What was the accuracy of the sentiment_model that we learned using 
# all the word counts in the IPython Notebook above from the lectures? 
# What is the accuracy majority class classifier on this task? How do 
# you compare the different learned models with the baseline approach 
# where we are just predicting the majority class?

selected_words_model.evaluate(test_data1, metric='roc_curve')



{'roc_curve': Columns:
 	threshold	float
 	fpr	float
 	tpr	float
 	p	int
 	n	int
 
 Rows: 1001
 
 Data:
 +------------------+----------------+-------------------+-------+------+
 |    threshold     |      fpr       |        tpr        |   p   |  n   |
 +------------------+----------------+-------------------+-------+------+
 |       0.0        |      0.0       | 3.57730557344e-05 | 27954 | 5323 |
 | 0.0010000000475  |      1.0       |   0.999964226944  | 27954 | 5323 |
 | 0.00200000009499 | 0.999812136014 |   0.999928453889  | 27954 | 5323 |
 | 0.00300000002608 | 0.999812136014 |   0.999928453889  | 27954 | 5323 |
 | 0.00400000018999 | 0.999624272027 |   0.999928453889  | 27954 | 5323 |
 | 0.00499999988824 | 0.999624272027 |   0.999928453889  | 27954 | 5323 |
 | 0.00600000005215 | 0.999624272027 |   0.999892680833  | 27954 | 5323 |
 | 0.00700000021607 | 0.999624272027 |   0.999892680833  | 27954 | 5323 |
 | 0.00800000037998 | 0.999624272027 |   0.999892680833  | 27954 | 5323 |
 | 0.008

In [85]:
selected_words_model.show(view='Evaluation')

#4. Interpreting the difference in performance between the models
##To understand why the model with all word counts performs better than the one with only the selected_words, we will now examine the reviews for a particular product.

In [89]:
# We will investigate a product named ‘Baby Trend Diaper Champ’. 
# (This is a trash can for soiled baby diapers, which keeps the smell contained.)

diaper_champ_reviews = products_new[products_new['name'] == 'Baby Trend Diaper Champ']
diaper_champ_reviews.head()

Columns:
	name	str
	review	str
	rating	float
	word_count	dict
	sentiment	int
	selected_words	dict
	awesome	int
	great	int
	fantastic	int
	amazing	int
	love	int
	horrible	int
	bad	int
	terrible	int
	awful	int
	wow	int
	hate	int

Rows: 10

Data:
+-------------------------+-------------------------------+--------+
|           name          |             review            | rating |
+-------------------------+-------------------------------+--------+
| Baby Trend Diaper Champ | Ok - newsflash.  Diapers a... |  4.0   |
| Baby Trend Diaper Champ | My husband and I selected ... |  1.0   |
| Baby Trend Diaper Champ | Excellent diaper disposal ... |  5.0   |
| Baby Trend Diaper Champ | We love our diaper champ. ... |  5.0   |
| Baby Trend Diaper Champ | Two girlfriends and two fa... |  5.0   |
| Baby Trend Diaper Champ | I waited to review this un... |  4.0   |
| Baby Trend Diaper Champ | I have had a diaper genie ... |  1.0   |
| Baby Trend Diaper Champ | I originally put this item... |  5.0   |
| Baby Trend Diaper Champ | I am so glad I got the Dia... |  5.0   |
| Baby Trend Diaper Champ | We had 2 diaper Genie's bo... |  4.0   |
+-------------------------+-------------------------------+--------+
+-------------------------------+-----------+-------------------------------+---------+
|           word_count          | sentiment |         selected_words        | awesome |
+-------------------------------+-----------+-------------------------------+---------+
| {'son': 1, 'just': 2, 'les... |     1     | {'great': 0, 'love': 0, 'a... |    0    |
| {'material)': 1, 'bags,': ... |     0     | {'great': 0, 'love': 0, 'a... |    0    |
| {'control': 1, 'am': 1, 'i... |     1     | {'great': 0, 'love': 0, 'a... |    0    |
| {'and': 3, 'over.': 1, 'al... |     1     | {'great': 0, 'love': 1, 'a... |    0    |
| {'just': 1, '-': 3, 'both'... |     1     | {'great': 0, 'love': 0, 'a... |    0    |
| {'lysol': 1, 'all': 1, 'mo... |     1     | {'great': 0, 'love': 0, 'a... |    0    |
| {'all': 1, 'bags.': 1, 'ju... |     0     | {'great': 0, 'love': 0, 'a... |    0    |
| {'lysol': 1, 'all': 2, 'ba... |     1     | {'great': 0, 'love': 0, 'a... |    0    |
| {'and': 2, 'all': 1, 'just... |     1     | {'great': 0, 'love': 0, 'a... |    0    |
| {'hand.': 1, 'both': 1, '(... |     1     | {'great': 0, 'love': 2, 'a... |    0    |
+-------------------------------+-----------+-------------------------------+---------+
+-------+-----------+---------+------+----------+-----+----------+-------+-----+
| great | fantastic | amazing | love | horrible | bad | terrible | awful | wow |
+-------+-----------+---------+------+----------+-----+----------+-------+-----+
|   0   |     0     |    0    |  0   |    0     |  0  |    0     |   0   |  0  |
|   0   |     0     |    0    |  0   |    0     |  0  |    0     |   0   |  0  |
|   0   |     0     |    0    |  0   |    0     |  0  |    0     |   0   |  0  |
|   0   |     0     |    0    |  1   |    0     |  0  |    0     |   0   |  0  |
|   0   |     0     |    0    |  0   |    1     |  0  |    0     |   0   |  0  |
|   0   |     0     |    0    |  0   |    0     |  1  |    0     |   0   |  0  |
|   0   |     0     |    0    |  0   |    0     |  0  |    0     |   0   |  0  |
|   0   |     0     |    0    |  0   |    0     |  0  |    0     |   0   |  0  |
|   0   |     0     |    0    |  0   |    0     |  0  |    0     |   0   |  0  |
|   0   |     0     |    0    |  2   |    0     |  0  |    0     |   0   |  0  |
+-------+-----------+---------+------+----------+-----+----------+-------+-----+
+------+
| hate |
+------+
|  0   |
|  0   |
|  0   |
|  0   |
|  0   |
|  0   |
|  0   |
|  0   |
|  0   |
|  0   |
+------+
[10 rows x 17 columns]

In [93]:
# Just like we did for the reviews for the giraffe toy in the IPython Notebook in the lecture video, 
# before we start our analysis you should select all reviews where the product name is ‘Baby Trend Diaper Champ’. 
# Let’s call this table diaper_champ_reviews.
# Again, just as in the video, use the sentiment_model to predict the sentiment of each review in 
# diaper_champ_reviews and sort the results according to their ‘predicted_sentiment’
# What is the ‘predicted_sentiment’ for the most positive review for ‘Baby Trend Diaper Champ’ 
# according to the sentiment_model from the IPython Notebook from lecture? 

diaper_champ_reviews['predicted_sentiment'] = sentiment_model.predict(diaper_champ_reviews, output_type='probability')
diaper_champ_reviews = diaper_champ_reviews.sort('predicted_sentiment', ascending=False)
diaper_champ_reviews.head()

Columns:
	name	str
	review	str
	rating	float
	word_count	dict
	sentiment	int
	selected_words	dict
	awesome	int
	great	int
	fantastic	int
	amazing	int
	love	int
	horrible	int
	bad	int
	terrible	int
	awful	int
	wow	int
	hate	int
	predicted_sentiment	float

Rows: 10

Data:
+-------------------------+-------------------------------+--------+
|           name          |             review            | rating |
+-------------------------+-------------------------------+--------+
| Baby Trend Diaper Champ | Baby Luke can turn a clean... |  5.0   |
| Baby Trend Diaper Champ | I LOOOVE this diaper pail!... |  5.0   |
| Baby Trend Diaper Champ | We researched all of the d... |  4.0   |
| Baby Trend Diaper Champ | My baby is now 8 months an... |  5.0   |
| Baby Trend Diaper Champ | This is absolutely, by far... |  5.0   |
| Baby Trend Diaper Champ | Diaper Champ or Diaper Gen... |  5.0   |
| Baby Trend Diaper Champ | Wow!  This is fabulous.  I... |  5.0   |
| Baby Trend Diaper Champ | I originally put this item... |  5.0   |
| Baby Trend Diaper Champ | Two girlfriends and two fa... |  5.0   |
| Baby Trend Diaper Champ | I am one of those super-cr... |  5.0   |
+-------------------------+-------------------------------+--------+
+-------------------------------+-----------+-------------------------------+---------+
|           word_count          | sentiment |         selected_words        | awesome |
+-------------------------------+-----------+-------------------------------+---------+
| {'all': 1, 'less': 1, "fri... |     1     | {'great': 0, 'love': 0, 'a... |    0    |
| {'just': 1, 'over': 1, 'rw... |     1     | {'great': 0, 'love': 1, 'a... |    0    |
| {'all': 2, 'just': 4, "don... |     1     | {'great': 0, 'love': 0, 'a... |    0    |
| {"don't": 1, 'able': 2, 'o... |     1     | {'great': 2, 'love': 0, 'a... |    0    |
| {'just': 3, 'money': 1, 's... |     1     | {'great': 0, 'love': 2, 'a... |    0    |
| {'son': 2, 'all': 1, 'bags... |     1     | {'great': 0, 'love': 0, 'a... |    0    |
| {'and': 4, 'this': 3, 'sti... |     1     | {'great': 0, 'love': 0, 'a... |    0    |
| {'lysol': 1, 'all': 2, 'ba... |     1     | {'great': 0, 'love': 0, 'a... |    0    |
| {'just': 1, '-': 3, 'both'... |     1     | {'great': 0, 'love': 0, 'a... |    0    |
| {'all': 1, 'humid': 1, 'ju... |     1     | {'great': 0, 'love': 1, 'a... |    0    |
+-------------------------------+-----------+-------------------------------+---------+
+-------+-----------+---------+------+----------+-----+----------+-------+-----+
| great | fantastic | amazing | love | horrible | bad | terrible | awful | wow |
+-------+-----------+---------+------+----------+-----+----------+-------+-----+
|   0   |     0     |    0    |  0   |    0     |  0  |    0     |   0   |  0  |
|   0   |     0     |    0    |  1   |    0     |  0  |    0     |   0   |  0  |
|   0   |     0     |    0    |  0   |    0     |  1  |    0     |   0   |  0  |
|   2   |     0     |    0    |  0   |    0     |  1  |    0     |   0   |  0  |
|   0   |     0     |    0    |  2   |    0     |  0  |    0     |   0   |  0  |
|   0   |     0     |    0    |  0   |    0     |  0  |    0     |   0   |  0  |
|   0   |     0     |    0    |  0   |    0     |  0  |    0     |   0   |  0  |
|   0   |     0     |    0    |  0   |    0     |  0  |    0     |   0   |  0  |
|   0   |     0     |    0    |  0   |    1     |  0  |    0     |   0   |  0  |
|   0   |     0     |    0    |  1   |    0     |  0  |    0     |   0   |  0  |
+-------+-----------+---------+------+----------+-----+----------+-------+-----+
+------+---------------------+
| hate | predicted_sentiment |
+------+---------------------+
|  0   |    0.999999937267   |
|  0   |    0.999999917406   |
|  0   |    0.999999899509   |
|  0   |    0.999999836182   |
|  0   |    0.999999824745   |
|  0   |    0.999999759315   |
|  0   |    0.999999692111   |
|  0   |    0.999999642488   |
|  0   |    0.999999604504   |
|  0   |    0.99999948680

In [94]:
diaper_champ_reviews[0]['review']

'Baby Luke can turn a clean diaper to a dirty diaper in 3 seconds flat. The diaper champ turns the smelly diaper into "what diaper smell" in less time than that. I hesitated and wondered what I REALLY needed for the nursery. This is one of the best purchases we made. The champ, the baby bjorn, fluerville diaper bag, and graco pack and play bassinet all vie for the best baby purchase.Great product, easy to use, economical, effective, absolutly fabulous.UpdateI knew that I loved the champ, and useing the diaper genie at a friend\'s house REALLY reinforced that!! There is no comparison, the chanp is easy and smell free, the genie was difficult to use one handed (which is absolutly vital if you have a little one on a changing pad) and there was a deffinite odor eminating from the genieplus we found that the quick tie garbage bags where the ties are integrated into the bag work really well because there isn\'t any added bulk around the sealing edge of the champ.'

In [95]:
diaper_champ_reviews[-1]['review']

'My husband and I selected the Diaper "Champ" mainly because you can use ordinary trash bags and not be roped into buying the specialty refill bags, and it was moderately priced (a little less than the Diaper Dekor). It also seemed that the reviews of this product were generally more positive...The positives are:1. You can use any trash bag2. Easy to use and refillThe negatives are:1. The bag doesn\'t seal around the dirty diapers, so when it comes time to refill the bag, it\'s just like opening a regular trash can. Smells like the Champ is trying to knock YOU out with odor!2. The plastic seems to smell, ie. You put a dirty diaper in the hole, and flip the handle to dump the diaper into the champ. That "side" of the plastic dumper-thingie is in contact with the air inside the dirty diaper changer, so when you flip it over the next time to dispose of another diaper, you smell the last 8 diapers you put in there...pretty gross.3. The "odor seal" (some soft material) really seems to retai

In [97]:
# Now use the selected_words_model you learned using just the selected_words 
# to predict the sentiment most positive review you found above. 
# Hint: if you sorted the diaper_champ_reviews in descending order 
# (from most positive to most negative), this command will be helpful to make the prediction you need:
# selected_words_model.predict(diaper_champ_reviews[0:1], output_type='probability')

diaper_champ_reviews['selected_sentiment'] = selected_words_model.predict(diaper_champ_reviews, output_type='probability')
diaper_champ_reviews = diaper_champ_reviews.sort('selected_sentiment', ascending=False)
diaper_champ_reviews.head()

Columns:
	name	str
	review	str
	rating	float
	word_count	dict
	sentiment	int
	selected_words	dict
	awesome	int
	great	int
	fantastic	int
	amazing	int
	love	int
	horrible	int
	bad	int
	terrible	int
	awful	int
	wow	int
	hate	int
	predicted_sentiment	float
	selected_sentiment	float

Rows: 10

Data:
+-------------------------+-------------------------------+--------+
|           name          |             review            | rating |
+-------------------------+-------------------------------+--------+
| Baby Trend Diaper Champ | I LOVE LOVE LOVE this prod... |  4.0   |
| Baby Trend Diaper Champ | Love it, love it, love it!... |  5.0   |
| Baby Trend Diaper Champ | I received my Diaper Champ... |  5.0   |
| Baby Trend Diaper Champ | I love this diaper pale an... |  5.0   |
| Baby Trend Diaper Champ | Works great - no smells.  ... |  5.0   |
| Baby Trend Diaper Champ | I love this diaper pail!  ... |  5.0   |
| Baby Trend Diaper Champ | I love this diaper pail. I... |  4.0   |
| Baby Trend Diaper Champ | I've worked with kids more... |  5.0   |
| Baby Trend Diaper Champ | This is absolutely, by far... |  5.0   |
| Baby Trend Diaper Champ | I have a 10 year old daugh... |  5.0   |
+-------------------------+-------------------------------+--------+
+-------------------------------+-----------+-------------------------------+---------+
|           word_count          | sentiment |         selected_words        | awesome |
+-------------------------------+-----------+-------------------------------+---------+
| {'rating': 1, 'contacted':... |     1     | {'great': 1, 'love': 3, 'a... |    0    |
| {'instead': 1, 'all': 1, '... |     1     | {'great': 0, 'love': 3, 'a... |    0    |
| {'bags.': 1, "don't": 1, '... |     1     | {'great': 0, 'love': 3, 'a... |    0    |
| {'and': 3, 'love': 1, 'use... |     1     | {'great': 2, 'love': 1, 'a... |    0    |
| {'and': 2, 'bags.': 1, 'ga... |     1     | {'great': 2, 'love': 1, 'a... |    0    |
| {'and': 3, 'love': 2, 'bei... |     1     | {'great': 0, 'love': 2, 'a... |    0    |
| {'and': 1, 'old': 1, 'extr... |     1     | {'great': 0, 'love': 2, 'a... |    0    |
| {'and': 4, 'genies': 1, 'a... |     1     | {'great': 0, 'love': 2, 'a... |    0    |
| {'just': 3, 'money': 1, 's... |     1     | {'great': 0, 'love': 2, 'a... |    0    |
| {'lysol': 1, 'all': 1, 'ju... |     1     | {'great': 0, 'love': 2, 'a... |    0    |
+-------------------------------+-----------+-------------------------------+---------+
+-------+-----------+---------+------+----------+-----+----------+-------+-----+
| great | fantastic | amazing | love | horrible | bad | terrible | awful | wow |
+-------+-----------+---------+------+----------+-----+----------+-------+-----+
|   1   |     0     |    0    |  3   |    0     |  0  |    0     |   0   |  0  |
|   0   |     0     |    0    |  3   |    0     |  0  |    0     |   0   |  0  |
|   0   |     0     |    0    |  3   |    0     |  0  |    0     |   0   |  0  |
|   2   |     0     |    0    |  1   |    0     |  0  |    0     |   0   |  0  |
|   2   |     0     |    0    |  1   |    0     |  0  |    0     |   0   |  0  |
|   0   |     0     |    0    |  2   |    0     |  0  |    0     |   0   |  0  |
|   0   |     0     |    0    |  2   |    0     |  0  |    0     |   0   |  0  |
|   0   |     0     |    0    |  2   |    0     |  0  |    0     |   0   |  0  |
|   0   |     0     |    0    |  2   |    0     |  0  |    0     |   0   |  0  |
|   0   |     0     |    0    |  2   |    0     |  0  |    0     |   0   |  0  |
+-------+-----------+---------+------+----------+-----+----------+-------+-----+
+------+---------------------+--------------------+
| hate | predicted_sentiment | selected_sentiment |
+------+---------------------+--------------------+
|  0   |    0.999993652036   |   0.998423414594   |
|  0   |    0.985732101571   |   0.996192539732   |
|  0   |    0.999301330286   |   0.996192539732   |
|  0   |    0.983086548255   |   0.989387539605   |
|  0   

In [98]:
# Why is the predicted_sentiment for the most positive review found using the model with 
# all word counts (sentiment_model) much more positive than the one using only the 
# selected_words (selected_words_model)? Hint: examine the text of this review, 
# the extracted word counts for all words, and the word counts for each of the selected_words, 
# and you will see what each model used to make its prediction. 
diaper_champ_reviews[0]['review']

'I LOVE LOVE LOVE this product! It is SO much easier to use than the Diaper Genie, (you need a PHD in poopy to figure out how to use the darn thing!) and it even takes the same bags as my kitchen trash can, shich is super convenient, and cost efficient as I can buy them in bulk.The only reason for not rating it a 5 star was that I did have one small problem with it. The foam gasket in the barrell which keeps the poopy smell inside the unit ripped somehow, and it got VERY stinky. HOWEVER, I contacted the manufacturer though their website, and received an email back the same day stating that this was unusual, and that replacement gaskets were on their way to me. They arrived inside of a week and after replacing, it works great again! (They even sent me extras should it happen again)I HIGHLY reccomend this diaper pail over ANY competitors, you will not be sorry!'

In [99]:
diaper_champ_reviews[-1]['review']

"The Diaper Champ is TERRIBLE at keeping the smelly diapers from only smelling in the container.  Our baby's room was constantly stinky (due to the Diaper Champ, not the baby!), and we were having to empty the container almost daily.  What's the point of having a diaper disposal system if you can't dispose of diapers efficiently?  Please don't buy this product unless you enjoy smelling those dirty diapers.  The Diaper Champ just doesn't work."

In [102]:
diaper_champ_reviews = diaper_champ_reviews.sort('predicted_sentiment', ascending=False)
diaper_champ_reviews.head()

Columns:
	name	str
	review	str
	rating	float
	word_count	dict
	sentiment	int
	selected_words	dict
	awesome	int
	great	int
	fantastic	int
	amazing	int
	love	int
	horrible	int
	bad	int
	terrible	int
	awful	int
	wow	int
	hate	int
	predicted_sentiment	float
	selected_sentiment	float

Rows: 10

Data:
+-------------------------+-------------------------------+--------+
|           name          |             review            | rating |
+-------------------------+-------------------------------+--------+
| Baby Trend Diaper Champ | Baby Luke can turn a clean... |  5.0   |
| Baby Trend Diaper Champ | I LOOOVE this diaper pail!... |  5.0   |
| Baby Trend Diaper Champ | We researched all of the d... |  4.0   |
| Baby Trend Diaper Champ | My baby is now 8 months an... |  5.0   |
| Baby Trend Diaper Champ | This is absolutely, by far... |  5.0   |
| Baby Trend Diaper Champ | Diaper Champ or Diaper Gen... |  5.0   |
| Baby Trend Diaper Champ | Wow!  This is fabulous.  I... |  5.0   |
| Baby Trend Diaper Champ | I originally put this item... |  5.0   |
| Baby Trend Diaper Champ | Two girlfriends and two fa... |  5.0   |
| Baby Trend Diaper Champ | I am one of those super-cr... |  5.0   |
+-------------------------+-------------------------------+--------+
+-------------------------------+-----------+-------------------------------+---------+
|           word_count          | sentiment |         selected_words        | awesome |
+-------------------------------+-----------+-------------------------------+---------+
| {'all': 1, 'less': 1, "fri... |     1     | {'great': 0, 'love': 0, 'a... |    0    |
| {'just': 1, 'over': 1, 'rw... |     1     | {'great': 0, 'love': 1, 'a... |    0    |
| {'all': 2, 'just': 4, "don... |     1     | {'great': 0, 'love': 0, 'a... |    0    |
| {"don't": 1, 'able': 2, 'o... |     1     | {'great': 2, 'love': 0, 'a... |    0    |
| {'just': 3, 'money': 1, 's... |     1     | {'great': 0, 'love': 2, 'a... |    0    |
| {'son': 2, 'all': 1, 'bags... |     1     | {'great': 0, 'love': 0, 'a... |    0    |
| {'and': 4, 'this': 3, 'sti... |     1     | {'great': 0, 'love': 0, 'a... |    0    |
| {'lysol': 1, 'all': 2, 'ba... |     1     | {'great': 0, 'love': 0, 'a... |    0    |
| {'just': 1, '-': 3, 'both'... |     1     | {'great': 0, 'love': 0, 'a... |    0    |
| {'all': 1, 'humid': 1, 'ju... |     1     | {'great': 0, 'love': 1, 'a... |    0    |
+-------------------------------+-----------+-------------------------------+---------+
+-------+-----------+---------+------+----------+-----+----------+-------+-----+
| great | fantastic | amazing | love | horrible | bad | terrible | awful | wow |
+-------+-----------+---------+------+----------+-----+----------+-------+-----+
|   0   |     0     |    0    |  0   |    0     |  0  |    0     |   0   |  0  |
|   0   |     0     |    0    |  1   |    0     |  0  |    0     |   0   |  0  |
|   0   |     0     |    0    |  0   |    0     |  1  |    0     |   0   |  0  |
|   2   |     0     |    0    |  0   |    0     |  1  |    0     |   0   |  0  |
|   0   |     0     |    0    |  2   |    0     |  0  |    0     |   0   |  0  |
|   0   |     0     |    0    |  0   |    0     |  0  |    0     |   0   |  0  |
|   0   |     0     |    0    |  0   |    0     |  0  |    0     |   0   |  0  |
|   0   |     0     |    0    |  0   |    0     |  0  |    0     |   0   |  0  |
|   0   |     0     |    0    |  0   |    1     |  0  |    0     |   0   |  0  |
|   0   |     0     |    0    |  1   |    0     |  0  |    0     |   0   |  0  |
+-------+-----------+---------+------+----------+-----+----------+-------+-----+
+------+---------------------+--------------------+
| hate | predicted_sentiment | selected_sentiment |
+------+---------------------+--------------------+
|  0   |    0.999999937267   |   0.796940851291   |
|  0   |    0.999999917406   |   0.940876393428   |
|  0   |    0.999999899509   |    0.5942241719    |
|  0   |    0.999999836182   |   0.895606298305   |
|  0   